In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from Data_cleaning import *

# Xử lý dữ liệu 

In [2]:
#import books, users, ratings clean data
books, users, ratings = get_clean_data(path="/content/drive/My Drive/Colab Notebooks/recommendation_system/data/")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:28

In [3]:
books.head()

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
#number of ratings received by each book
usersPerIsbn = ratings['isbn'].value_counts()
usersPerIsbn

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
0684185008       1
0959365915       1
187842484X       1
0933377215       1
8434866196       1
Name: isbn, Length: 340556, dtype: int64

In [6]:
books_10 = books[books['isbn'].isin(usersPerIsbn[usersPerIsbn>10].index)]
books_10 = books_10.reset_index(drop=True)
# books_10 = books[:50000]

# Chỉ sử dụng `Book-Title`

In [7]:
vectorizer = TfidfVectorizer(analyzer='word')
#build book-title tfidf matrix
tfidf_matrix = vectorizer.fit_transform(books_10['title'])
tfidf_matrix.shape

(15452, 10972)

In [18]:
#Function to get the most similar books
indices_n = pd.Series(books_10['isbn']).to_dict()
inddict = dict((v,k) for k,v in indices_n.items())
def recommend_cosine(isbn, method):
    id = inddict[isbn]
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    print(books_10.iloc[similarity_scores[0][0]]['isbn'])
    similarity_scores = similarity_scores[1:6]
    
    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10.iloc[books_index]

def recommend_euclidean (isbn, method):
    id = inddict[isbn]
    # Get the pairwise similarity scores of all books compared that book,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1])
    print(books_10.iloc[similarity_scores[0][0]]['isbn'])
    similarity_scores = similarity_scores[1:6]

    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10.iloc[books_index]

In [9]:
isbn_test = "0971880107"
books_10.iloc[inddict[isbn_test]]

isbn                                                0971880107
title                                              Wild Animus
author                                            Rich Shapero
pub_year                                                  2004
publisher                                              Too Far
url_s        http://images.amazon.com/images/P/0971880107.0...
url_m        http://images.amazon.com/images/P/0971880107.0...
url_l        http://images.amazon.com/images/P/0971880107.0...
Name: 7, dtype: object

## Cosine similarity

In [10]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)


In [11]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
recommend_cosine(isbn_test, cosine_similarity).head()

7


,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
5309,0812512774,Wild,Whitley Strieber,1991.0,Tor Books,http://images.amazon.com/images/P/0812512774.0...,http://images.amazon.com/images/P/0812512774.0...,http://images.amazon.com/images/P/0812512774.0...
13062,0515132292,Wild,Lori Foster,2002.0,Jove Books,http://images.amazon.com/images/P/0515132292.0...,http://images.amazon.com/images/P/0515132292.0...,http://images.amazon.com/images/P/0515132292.0...
15153,0684867982,Wild Life : A Novel,Molly Gloss,2000.0,Simon &amp; Schuster,http://images.amazon.com/images/P/0684867982.0...,http://images.amazon.com/images/P/0684867982.0...,http://images.amazon.com/images/P/0684867982.0...
6152,0345423038,The Wild Road,Gabriel King,1999.0,Del Rey Books,http://images.amazon.com/images/P/0345423038.0...,http://images.amazon.com/images/P/0345423038.0...,http://images.amazon.com/images/P/0345423038.0...
8851,0553297619,The Wild Rose,Doris Mortman,1992.0,Bantam Books,http://images.amazon.com/images/P/0553297619.0...,http://images.amazon.com/images/P/0553297619.0...,http://images.amazon.com/images/P/0553297619.0...


## Euclidean Distance

In [12]:
euclidean = euclidean_distances(tfidf_matrix)
recommend_euclidean(isbn_test, euclidean)


7


,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
5309,0812512774,Wild,Whitley Strieber,1991.0,Tor Books,http://images.amazon.com/images/P/0812512774.0...,http://images.amazon.com/images/P/0812512774.0...,http://images.amazon.com/images/P/0812512774.0...
13062,0515132292,Wild,Lori Foster,2002.0,Jove Books,http://images.amazon.com/images/P/0515132292.0...,http://images.amazon.com/images/P/0515132292.0...,http://images.amazon.com/images/P/0515132292.0...
2050,0151010633,Q,Luther Blissett,2004.0,Harcourt,http://images.amazon.com/images/P/0151010633.0...,http://images.amazon.com/images/P/0151010633.0...,http://images.amazon.com/images/P/0151010633.0...
4208,0452281881,e,Matt Beaumont,2000.0,Plume Books,http://images.amazon.com/images/P/0452281881.0...,http://images.amazon.com/images/P/0452281881.0...,http://images.amazon.com/images/P/0452281881.0...
5043,0449216527,S.,John Updike,1994.0,Fawcett Books,http://images.amazon.com/images/P/0449216527.0...,http://images.amazon.com/images/P/0449216527.0...,http://images.amazon.com/images/P/0449216527.0...


# Sử dụng 3 trường `Book-Title`, `Book-Author`, `Publisher`


In [19]:
books_10['all_content'] = books_10['title'] + books_10['author'] + books_10['publisher']
books_10.head(5)

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l,all_content
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara CallanRichard Bruce WrightHarperFlamingo...
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...
2,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,The Kitchen God's WifeAmy TanPutnam Pub Group
3,0440234743,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,The TestamentJohn GrishamDell
4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994.0,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,Beloved (Plume Contemporary Fiction)Toni Morri...


In [14]:
tfidf_all_content = vectorizer.fit_transform(books_10['all_content'])
tfidf_all_content.shape

(15452, 25806)

In [17]:
isbn_test = "0971880107"
books_10.iloc[inddict[isbn_test]]

isbn                                                  0971880107
title                                                Wild Animus
author                                              Rich Shapero
pub_year                                                    2004
publisher                                                Too Far
url_s          http://images.amazon.com/images/P/0971880107.0...
url_m          http://images.amazon.com/images/P/0971880107.0...
url_l          http://images.amazon.com/images/P/0971880107.0...
all_content                       Wild AnimusRich ShaperoToo Far
Name: 7, dtype: object

## Cosine similarity

In [20]:
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_similarity_all_content = linear_kernel(tfidf_all_content, tfidf_all_content)
recommend_cosine(isbn_test, cosine_similarity_all_content)

isbn                                                  0971880107
title                                                Wild Animus
author                                              Rich Shapero
pub_year                                                    2004
publisher                                                Too Far
url_s          http://images.amazon.com/images/P/0971880107.0...
url_m          http://images.amazon.com/images/P/0971880107.0...
url_l          http://images.amazon.com/images/P/0971880107.0...
all_content                       Wild AnimusRich ShaperoToo Far
Name: 7, dtype: object


,isbn,title,author,pub_year,publisher,url_s,url_m,url_l,all_content
5606,0836220870,"Hound Of The Far Side (Far Side Books, Collect...",Gary Larson,1987.0,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836220870.0...,http://images.amazon.com/images/P/0836220870.0...,http://images.amazon.com/images/P/0836220870.0...,"Hound Of The Far Side (Far Side Books, Collect..."
15408,0671027077,Far Harbor,JoAnn Ross,2000.0,Pocket,http://images.amazon.com/images/P/0671027077.0...,http://images.amazon.com/images/P/0671027077.0...,http://images.amazon.com/images/P/0671027077.0...,Far HarborJoAnn RossPocket
12299,0836212002,The Far Side,Gary Larson,1982.0,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836212002.0...,http://images.amazon.com/images/P/0836212002.0...,http://images.amazon.com/images/P/0836212002.0...,The Far SideGary LarsonAndrews McMeel Publishing
5303,055312997X,The Far Pavillions,M. M. Kaye,1978.0,Bantam Books,http://images.amazon.com/images/P/055312997X.0...,http://images.amazon.com/images/P/055312997X.0...,http://images.amazon.com/images/P/055312997X.0...,The Far PavillionsM. M. KayeBantam Books
14307,0515120448,The Wild Baron,Catherine Coulter,1997.0,Jove Books,http://images.amazon.com/images/P/0515120448.0...,http://images.amazon.com/images/P/0515120448.0...,http://images.amazon.com/images/P/0515120448.0...,The Wild BaronCatherine CoulterJove Books


---

## Euclidean Distance

In [21]:
euclidean_all_contend = euclidean_distances(tfidf_all_content)
recommend_euclidean(isbn_test, euclidean_all_contend)

isbn                                                  0971880107
title                                                Wild Animus
author                                              Rich Shapero
pub_year                                                    2004
publisher                                                Too Far
url_s          http://images.amazon.com/images/P/0971880107.0...
url_m          http://images.amazon.com/images/P/0971880107.0...
url_l          http://images.amazon.com/images/P/0971880107.0...
all_content                       Wild AnimusRich ShaperoToo Far
Name: 7, dtype: object


,isbn,title,author,pub_year,publisher,url_s,url_m,url_l,all_content
5606,0836220870,"Hound Of The Far Side (Far Side Books, Collect...",Gary Larson,1987.0,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836220870.0...,http://images.amazon.com/images/P/0836220870.0...,http://images.amazon.com/images/P/0836220870.0...,"Hound Of The Far Side (Far Side Books, Collect..."
15408,0671027077,Far Harbor,JoAnn Ross,2000.0,Pocket,http://images.amazon.com/images/P/0671027077.0...,http://images.amazon.com/images/P/0671027077.0...,http://images.amazon.com/images/P/0671027077.0...,Far HarborJoAnn RossPocket
12299,0836212002,The Far Side,Gary Larson,1982.0,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836212002.0...,http://images.amazon.com/images/P/0836212002.0...,http://images.amazon.com/images/P/0836212002.0...,The Far SideGary LarsonAndrews McMeel Publishing
5303,055312997X,The Far Pavillions,M. M. Kaye,1978.0,Bantam Books,http://images.amazon.com/images/P/055312997X.0...,http://images.amazon.com/images/P/055312997X.0...,http://images.amazon.com/images/P/055312997X.0...,The Far PavillionsM. M. KayeBantam Books
14307,0515120448,The Wild Baron,Catherine Coulter,1997.0,Jove Books,http://images.amazon.com/images/P/0515120448.0...,http://images.amazon.com/images/P/0515120448.0...,http://images.amazon.com/images/P/0515120448.0...,The Wild BaronCatherine CoulterJove Books


---